<a href="https://colab.research.google.com/github/Jessica1842553/MCDdm2026/blob/main/Tarea1_DM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tarea 1 - **Creación y operaciones básicas con PySpark**
### Jessica Lizeth Hernández Bracho - 1842553

### **MovieLens 10M 🎥**

Se eligió el conjunto de datos MovieLens 10M, el cual contiene aproximadamente 10 millones de calificaciones realizadas por usuarios sobre películas.


*   Fuente: https://grouplens.org/datasets/movielens/10m/



In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession.builder.appName("T1MovieL").getOrCreate()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,306 B/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
25 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
# Descarga de MovieLens 10M dataset
!wget -q --show-progress http://files.grouplens.org/datasets/movielens/ml-10m.zip -P /content/
!unzip -q /content/ml-10m.zip -d /content/

ratings = spark.read \
    .option("delimiter", "::") \
    .option("header", "false") \
    .csv(
        "/content/ml-10M100K/ratings.dat",
        schema="userId INT, movieId INT, rating DOUBLE, timestamp LONG"
    )

movies = spark.read \
    .option("delimiter", "::") \
    .option("header", "false") \
    .csv(
        "/content/ml-10M100K/movies.dat",
        schema="movieId INT, title STRING, genres STRING"
    )

tags = spark.read \
    .option("delimiter", "::") \
    .option("header", "false") \
    .csv(
        "/content/ml-10M100K/tags.dat",
        schema="userId INT, movieId INT, tag STRING, timestamp LONG"
    )

ml-10m.zip          100%[===================>]  62.53M  16.4MB/s    in 4.9s    
replace /content/ml-10M100K/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/ml-10M100K/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/ml-10M100K/ratings.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/ml-10M100K/README.html? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/ml-10M100K/split_ratings.sh? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/ml-10M100K/tags.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [3]:
movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [4]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)



In [5]:
tags.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: long (nullable = true)



In [7]:
# Prueba
ratings.show(5)
movies.show(5)

ratings.count()
movies.count()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|    122|   5.0|838985046|
|     1|    185|   5.0|838983525|
|     1|    231|   5.0|838983392|
|     1|    292|   5.0|838983421|
|     1|    316|   5.0|838983392|
+------+-------+------+---------+
only showing top 5 rows
+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows


10681

In [8]:
df1 = ratings.join(movies, on="movieId", how="inner")

Para facilitar la interpretación de los resultados, se realizó un join entre las tablas de calificaciones y películas, permitiendo analizar directamente los títulos de las películas en lugar de trabajar únicamente con identificadores numéricos `(movieId)`.

### Filtrar datos

In [9]:
# ratings altos (≥ 4)
ratings_high = df1.filter(df1.rating >= 4)
ratings_high.select("title", "rating").show(10, truncate=False)

+-----------------------------+------+
|title                        |rating|
+-----------------------------+------+
|Boomerang (1992)             |5.0   |
|Net, The (1995)              |5.0   |
|Dumb & Dumber (1994)         |5.0   |
|Outbreak (1995)              |5.0   |
|Stargate (1994)              |5.0   |
|Star Trek: Generations (1994)|5.0   |
|Flintstones, The (1994)      |5.0   |
|Forrest Gump (1994)          |5.0   |
|Jungle Book, The (1994)      |5.0   |
|Lion King, The (1994)        |5.0   |
+-----------------------------+------+
only showing top 10 rows


In [10]:
ratings_user_1 = df1.filter(df1.userId == 1)
ratings_user_1.select("title", "rating").show(truncate=False)

+-----------------------------------------+------+
|title                                    |rating|
+-----------------------------------------+------+
|Boomerang (1992)                         |5.0   |
|Net, The (1995)                          |5.0   |
|Dumb & Dumber (1994)                     |5.0   |
|Outbreak (1995)                          |5.0   |
|Stargate (1994)                          |5.0   |
|Star Trek: Generations (1994)            |5.0   |
|Flintstones, The (1994)                  |5.0   |
|Forrest Gump (1994)                      |5.0   |
|Jungle Book, The (1994)                  |5.0   |
|Lion King, The (1994)                    |5.0   |
|Naked Gun 33 1/3: The Final Insult (1994)|5.0   |
|Speed (1994)                             |5.0   |
|Beverly Hills Cop III (1994)             |5.0   |
|Hot Shots! Part Deux (1993)              |5.0   |
|Jurassic Park (1993)                     |5.0   |
|Robin Hood: Men in Tights (1993)         |5.0   |
|Sleepless in Seattle (1993)   

### Estadísticas descriptivas básicas

In [11]:
ratings.describe().show()

+-------+------------------+-----------------+-----------------+--------------------+
|summary|            userId|          movieId|           rating|           timestamp|
+-------+------------------+-----------------+-----------------+--------------------+
|  count|          10000054|         10000054|         10000054|            10000054|
|   mean| 35869.85940925919|4120.291476526027|3.512421932921562|1.0326063546983086E9|
| stddev|20585.337354675758|8938.402117920734|1.060418471626362|1.1596396220252669E8|
|    min|                 1|                1|              0.5|           789652009|
|    max|             71567|            65133|              5.0|          1231131736|
+-------+------------------+-----------------+-----------------+--------------------+



### Operaciones aritméticas

In [12]:
ratings.groupBy("rating").count().orderBy("rating").show()

+------+-------+
|rating|  count|
+------+-------+
|   0.5|  94988|
|   1.0| 384180|
|   1.5| 118278|
|   2.0| 790306|
|   2.5| 370178|
|   3.0|2356676|
|   3.5| 879764|
|   4.0|2875850|
|   4.5| 585022|
|   5.0|1544812|
+------+-------+



In [13]:
# Promedio de rating por película
ratings.groupBy("movieId").avg("rating").withColumnRenamed("avg(rating)", "avg_rating").orderBy("avg_rating", ascending=False).show(10)

# Promedio de rating por película con nombres
from pyspark.sql.functions import avg, count
ratings_avg_movie = df1.groupBy("movieId", "title").agg(avg("rating").alias("avg_rating"),count("rating").alias("num_ratings"))
ratings_avg_movie.filter("num_ratings >= 100").orderBy("avg_rating", ascending=False).show(10, truncate=False)
# evitar películas con 1 solo rating

+-------+----------+
|movieId|avg_rating|
+-------+----------+
|  51209|       5.0|
|  42783|       5.0|
|  33264|       5.0|
|  53355|       5.0|
|  64275|       5.0|
|   5194|      4.75|
|  65001|      4.75|
|   4454|      4.75|
|  26048|      4.75|
|  26073|      4.75|
+-------+----------+
only showing top 10 rows
+-------+---------------------------------------------+-----------------+-----------+
|movieId|title                                        |avg_rating       |num_ratings|
+-------+---------------------------------------------+-----------------+-----------+
|318    |Shawshank Redemption, The (1994)             |4.457238321660348|31126      |
|858    |Godfather, The (1972)                        |4.415085293227011|19814      |
|50     |Usual Suspects, The (1995)                   |4.367142322253193|24037      |
|527    |Schindler's List (1993)                      |4.363482949916592|25777      |
|922    |Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)|4.321966205837174|3255  